<a href="https://colab.research.google.com/github/gsenas/deeplearning/blob/master/Fruit_Datagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En primer lugar, descargamos el dataset de fruits-360 de [github](https://github.com/Horea94/Fruit-Images-Dataset/archive/master.zip)

In [1]:
import tensorflow as tf
import os as os

_URL = 'https://github.com/Horea94/Fruit-Images-Dataset/archive/master.zip'

zip_file = tf.keras.utils.get_file(origin=_URL, fname='Fruit-Images-Dataset-master.zip', extract=True, cache_subdir='/content')

792723456/Unknown - 37s 0us/step

Establecemos los parámetros iniciales de tamaño de imagen y directorio donde buscar las fotos de frutas.

En este caso estamos aumentando las imágenes de prueba, rotando y modificando los tamaños para tener un dataset mayor por cada fruta

In [6]:
base_dir = './Fruit-Images-Dataset-master/Training'

IMAGE_SIZE = 224
BATCH_SIZE = 64

#datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation range=30,width_shift_range=0.3,height_shift_range=0.3,horizontal_flip=True,validation_split=0.1,fill_mode='nearest')

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    validation_split=0.1,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(base_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, subset='training') 

Found 60955 images belonging to 131 classes.


Nos aseguramos de que estamos obteniendo todas las frutas, y las imprimimos en el fichero de etiquetas

In [10]:
print(train_generator.class_indices)

labels = '\n'.join(sorted((train_generator.class_indices.keys())))

with open('labels.txt','w') as f:
  f.write(labels)

{'Apple Braeburn': 0, 'Apple Crimson Snow': 1, 'Apple Golden 1': 2, 'Apple Golden 2': 3, 'Apple Golden 3': 4, 'Apple Granny Smith': 5, 'Apple Pink Lady': 6, 'Apple Red 1': 7, 'Apple Red 2': 8, 'Apple Red 3': 9, 'Apple Red Delicious': 10, 'Apple Red Yellow 1': 11, 'Apple Red Yellow 2': 12, 'Apricot': 13, 'Avocado': 14, 'Avocado ripe': 15, 'Banana': 16, 'Banana Lady Finger': 17, 'Banana Red': 18, 'Beetroot': 19, 'Blueberry': 20, 'Cactus fruit': 21, 'Cantaloupe 1': 22, 'Cantaloupe 2': 23, 'Carambula': 24, 'Cauliflower': 25, 'Cherry 1': 26, 'Cherry 2': 27, 'Cherry Rainier': 28, 'Cherry Wax Black': 29, 'Cherry Wax Red': 30, 'Cherry Wax Yellow': 31, 'Chestnut': 32, 'Clementine': 33, 'Cocos': 34, 'Corn': 35, 'Corn Husk': 36, 'Cucumber Ripe': 37, 'Cucumber Ripe 2': 38, 'Dates': 39, 'Eggplant': 40, 'Fig': 41, 'Ginger Root': 42, 'Granadilla': 43, 'Grape Blue': 44, 'Grape Pink': 45, 'Grape White': 46, 'Grape White 2': 47, 'Grape White 3': 48, 'Grape White 4': 49, 'Grapefruit Pink': 50, 'Grapefrui

Preparamos un modelo secuencial que acumule varias capas. Al ejecutarlo no debería dar errores.

La densidad es 131 porque es el número de etiquetas diferentes que tenemos en el dataset

In [12]:
IMG_SHAPE=(IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3,activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(131,activation='softmax')
])

9412608/9406464 [==============================] - 0s 0us/step


Finalmente, compilamos la primera versión del modelo, y la ejecutamos

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs=10

history= model.fit(train_generator, epochs=epochs, steps_per_epoch=200)

Epoch 1/10
200/200 [==============================] - 143s 672ms/step - loss: 4.1359 - accuracy: 0.1234
Epoch 2/10
200/200 [==============================] - 132s 661ms/step - loss: 1.3439 - accuracy: 0.6261
Epoch 3/10
200/200 [==============================] - 132s 660ms/step - loss: 0.6724 - accuracy: 0.7975
Epoch 4/10
200/200 [==============================] - 132s 661ms/step - loss: 0.5262 - accuracy: 0.8326
Epoch 5/10
200/200 [==============================] - 134s 670ms/step - loss: 0.4126 - accuracy: 0.8699
Epoch 6/10
200/200 [==============================] - 135s 675ms/step - loss: 0.3434 - accuracy: 0.8916
Epoch 7/10
 40/200 [=====>........................] - ETA: 1:46 - loss: 0.3025 - accuracy: 0.8983

Una vez entrenado, exportamos el modelo, y lo descargamos

In [ ]:
saved_model_dir=''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite','wb') as f:
  f.write(tflite_model)

In [ ]:


from google.colab import files

files.download('model.tflite')
files.download('labels.txt')

